<h1 align="center">Analysis of the Amazon dataset of book reviews</h1>

The aim was to process the semantic analysis of Ukrainian texts, so the collection of positive and negative reviews was translated into Ukrainian using Google Translate.

To get started, we import all the libraries we need:

In [1]:
import os
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import xml.etree.ElementTree as ET

%matplotlib inline

Next, since our reviews are in XML format, we need to process each line to get the value from the tags.

**Note**: 
* `errors='ignore'`used to skip all possible incorrect characters in our documents;

* `readLines` return all lines in the file, as a list where each line is an item in the list object.

In [2]:
testRead = open('data/positive.review', 'r', encoding='utf8', errors='ignore').readlines()
for i in range(85):
    print(testRead[i])

<reviews>

<review>

<unique_id>

0375416811: owner_of_chick_lit: erin_oakes_ "avid_reader"

</unique_id>

<unique_id>

1521 рік

</unique_id>

<asin>

0375416811

</asin>

<product_name>

Щоденник Бріджит Джонс: Книги: Хелен Філдінг, Трейсі Беннет

</product_name>

<product_type>

книги

</product_type>

<product_type>

книги

</product_type>

<helpful>



</helpful>

<rating>

5.0

</rating>

<title>

Засновник Чик-літ

</title>

<date>

24 серпня 2006 р

</date>

<reviewer>

Ерін Оукс "завзятий читач"

</reviewer>

<reviewer_location>

Центральна ПА

</reviewer_location>

<review_text>

Бріджит Джонс, сучасна жінка, блискуча і не знає цього, схильна до нещасних випадків та безладів, але їм вдається вийти з них.



Це книга, яка розпочала все з лихоманки, що запалюється. Бріджит Джонс - мій герой!

</review_text>

</review>

<review>

<unique_id>

188105201X: відмінний_ресурс_для_принципів!: Onickre@mail.milwaukee.k12.wi.us

</unique_id>

<unique_id>

3294

</unique_id>

<asin>

1881

In our directory with training sets we have two files - with positive and negative reviews, each of these files contains 1000 unprocessed reviews. Let's check this:

In [3]:
pos_reviews = open('data/positive.review', 'r', encoding='utf8', errors='ignore').read()
neg_reviews = open('data/negative.review', 'r', encoding='utf8', errors='ignore').read()

pos_rev_tree = ET.fromstring(pos_reviews)
neg_rev_tree = ET.fromstring(neg_reviews)

pos_tags = pos_rev_tree.findall('review')
neg_tags = neg_rev_tree.findall('review')

print('\nNumber of Positive Reviews:', len(pos_tags),
      '\nNumber of Positive Reviews:', len(neg_tags))


Number of Positive Reviews: 1000 
Number of Positive Reviews: 1000


**Note:** 
* `fromstring()` parses XML from a string directly into an Element, which is the root element of the parsed tree;
* `findall() ` finds all elements with a tag which are direct children of the current element (`reviews`).

<h2 align="center">XML Parse</h2>

Now we will try to parse our XML file at a basic level. We will create a dictionary from the lists. Each list will contain certain tags and their content for single review. We already have a tree with positive and negative reviews, so all that remains is to parse it. But before that, let's list all the tags in our xml files so we don't have to repeat:

In [4]:
review_tags = ['unique_id', 'asin', 'product_name', 'helpful', 'rating', 'title',
               'date', 'reviewer', 'reviewer_location', 'review_text']

In [5]:
def parseXML(xml_reviews):
    reviews = {}
    count = 0

    for item in xml_reviews:
        count += 1
        rev_name = 'review' + str(count)
        reviews[rev_name] = [
            review_tags[0] + ' | ' + item.find(review_tags[0]).text,
            review_tags[1] + ' | ' + item.find(review_tags[1]).text,
            review_tags[2] + ' | ' + item.find(review_tags[2]).text,
            review_tags[3] + ' | ' + item.find(review_tags[3]).text,
            review_tags[4] + ' | ' + item.find(review_tags[4]).text,
            review_tags[5] + ' | ' + item.find(review_tags[5]).text,
            review_tags[6] + ' | ' + item.find(review_tags[6]).text,
            review_tags[7] + ' | ' + item.find(review_tags[7]).text,
            review_tags[8] + ' | ' + item.find(review_tags[8]).text,
            review_tags[9] + ' | ' + item.find(review_tags[9]).text
        ]

    return reviews

Now let's actually create dictionaries with the required values and check output:

In [6]:
posReviewsDict = parseXML(pos_tags)
negReviewsDict = parseXML(neg_tags)
posReviewsDict['review1']

['unique_id | \n0375416811: owner_of_chick_lit: erin_oakes_ "avid_reader"\n',
 'asin | \n0375416811\n',
 'product_name | \nЩоденник Бріджит Джонс: Книги: Хелен Філдінг, Трейсі Беннет\n',
 'helpful | \n\n',
 'rating | \n5.0\n',
 'title | \nЗасновник Чик-літ\n',
 'date | \n24 серпня 2006 р\n',
 'reviewer | \nЕрін Оукс "завзятий читач"\n',
 'reviewer_location | \nЦентральна ПА\n',
 'review_text | \nБріджит Джонс, сучасна жінка, блискуча і не знає цього, схильна до нещасних випадків та безладів, але їм вдається вийти з них.\n\nЦе книга, яка розпочала все з лихоманки, що запалюється. Бріджит Джонс - мій герой!\n']

Next, we will turn our reviews dictionaries into a Pandas DataFrame:

In [9]:
def dictToDataFrame(reviews_dict):
    # prepare our dataframe for the data
    df = pd.DataFrame(columns=review_tags)
    count = 0
    for val in reviews_dict.values():
        df.loc[count] = [
            val[0].split("|")[1].split("\n")[1], val[1].split("|")[1].split("\n")[1],
            val[2].split("|")[1].split("\n")[1], val[3].split("|")[1].split("\n")[1],
            val[4].split("|")[1].split("\n")[1], val[5].split("|")[1].split("\n")[1],
            val[6].split("|")[1].split("\n")[1], val[7].split("|")[1].split("\n")[1],
            val[8].split("|")[1].split("\n")[1], val[9].split("|")[1].split("\n")[1]
        ]

        count = count + 1

    return df

posBooks = dictToDataFrame(posReviewsDict)
negBooks = dictToDataFrame(negReviewsDict)

Let's check for correctness:

In [10]:
posBooks.head(n=3)

,unique_id,asin,product_name,helpful,rating,title,date,reviewer,reviewer_location,review_text
0,"0375416811: owner_of_chick_lit: erin_oakes_ ""a...",0375416811,"Щоденник Бріджит Джонс: Книги: Хелен Філдінг, ...",,5.0,Засновник Чик-літ,24 серпня 2006 р,"Ерін Оукс ""завзятий читач""",Центральна ПА,"Бріджит Джонс, сучасна жінка, блискуча і не зн..."
1,188105201X: відмінний_ресурс_для_принципів!: O...,188105201X,Лідерство та нова наука: вивчення організації ...,3 з 3,5.0,Чудовий ресурс для директорів!,6 липня 1999 р,ONICKRE@mail.milwaukee.k12.wi.us,"Мілуокі, штат Вісконсін",Я замовляю копії для всіх 23 директорів середн...
2,0634079190: fun_to_play: michael_p._nolan,0634079190,Avenue Q - мюзикл (аранжування на фортепіано /...,,5.0,Забавно грати,15 вересня 2005 р,Майкл П. Нолан,"Таллахассі, Флорида, США",Як випадковий гравець на фортепіано та фанат н...
